In [7]:
import os
print(os.listdir())
os.chdir('CS687/HW1_TimeSeries')

['.git', 'cs674_hw1(1).pdf', 'dtw.py', 'hw1_datasets', 'hw1_main.py', 'process_data.py', 'testing_code.ipynb', 'timeseries_kNN.py', '__pycache__']


FileNotFoundError: [WinError 3] The system cannot find the path specified: 'CS687/HW1_TimeSeries'

In [8]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from timeseries_kNN import timeseries_kNN
import os
from heapq import nsmallest
from random import shuffle
from datetime import datetime as dt
from multiprocessing.pool import ThreadPool
from multiprocessing import Pool, cpu_count
from threading import Thread
from sklearn.neighbors import NearestNeighbors, KNeighborsRegressor
from dask import delayed

In [14]:
dataset1_test = 'hw1_datasets/dataset1/test_normalized.csv'
test1_labels = 'hw1_datasets/dataset1/test_labels.csv'
dataset1_train = 'hw1_datasets/dataset1/train_normalized.csv'
train1_labels = 'hw1_datasets/dataset1/train_labels.csv'
train1 = pd.read_csv(dataset1_train)
test1 = pd.read_csv(dataset1_test, index_col=0)
train1.head()
test1.head()

,1,2,3,4,5,6,7,8,9,10,...,119,120,121,122,123,124,125,126,127,128
0,-1.186546,1.112753,1.499844,1.081663,1.425415,2.153796,1.518677,0.817144,1.196449,1.907417,...,0.920101,-0.007463,-0.218600,-0.494948,-0.103557,0.664229,0.945652,1.731896,1.228174,0.475412
1,0.407897,-0.583292,-0.739038,-1.673533,-0.065568,-0.018168,0.915209,-0.453858,-0.679295,-0.213708,...,-0.793742,-0.439977,-1.293828,1.242226,-0.620446,-0.492400,0.529914,-0.189582,0.256176,1.552341
2,-0.980611,-0.351831,-0.501440,-0.249934,-0.965480,-0.124003,-0.361499,-0.213201,-0.715557,-0.190532,...,-1.147009,-0.598236,-1.007310,-0.580817,-0.410211,-0.869793,-0.745043,-0.487621,-0.930685,-1.047306
3,-0.085029,0.091052,-1.140404,0.260809,0.360512,0.342949,-0.849694,-1.344433,-1.729363,-0.365701,...,-0.732892,0.455786,-0.905818,-0.034224,0.170775,0.470925,-0.173845,-0.166626,0.170123,0.135435
4,-1.053011,0.433085,-0.453950,0.146030,0.326497,-0.988377,-0.666821,-0.566277,-1.891093,-0.348393,...,3.153259,3.073105,3.398803,3.306304,4.264531,0.038569,-0.217056,-1.153955,-1.066324,-1.698859


In [11]:
from heapq import nsmallest, nlargest
from sklearn.neighbors import NearestNeighbors
import numpy as np
import pandas as pd
from sys import stdout
from random import shuffle
from datetime import datetime as dt
from threading import Thread
from multiprocessing.pool import ThreadPool
from multiprocessing import Process, cpu_count, Pool, Queue
import json
import math
from itertools import chain
import concurrent

In [ ]:
def kNN(self, a_chunk):
    '''
    Runs kNN on chunk of test set

    :param a_chunk:
    :return:
    '''
    predictions = []
    userid_list = []
    movieid_list = []
    userv_list = []
    prediction_id_list = []
    df = test1.iloc[a_chunk] # partition of test_df
    i = 0
    #name is None because of pickling issue with parallel
    for test_i in df.itertuples(name=None):
        if self.verbose:
            print('t0-t10 : {}'.format(test_i[:10]))
            print('ti: {}'.format(test_i))
        print(test_i[0])

        try:
            # time_series = self.data[users_that_reviewed_movie]
            euc_distances = self.data.apply(lambda test_i: self.dist(self.data, test_i[1:]), axis=1) # to each column
            print('euclidean : {}'.format(euc_distances))
            # using euclidean distance, returns list of tuples
            neighbors = nlargest(self.k, euc_distances.iteritems(), key=lambda x: x[1:])
            print('---after neighbors----')
            # if list of tuples is not empty
            neighbors = [(i,dist) for i,dist in neighbors]
            print('the neighbors are: '.format(neighbors))
            k_users, distances = zip(*neighbors)
            # k_users, distances = [i,dist for i,dist in neighbors]
            yhat = self.predict_yhat(test_i, k_users, distances)

            if test_i[0] % 2 == 0:
                if self.verbose: print('time is:{};\tindex: {};\tnumber of records: {};\t{} percent complete; '
                                       '\t rhat: {};\tthe review is:{}'.format(
                    self.get_time(), i, len(df), round(i/len(df), 2), yhat, test_i))
            predictions.append(yhat)
            prediction_id_list.append(test_i)
            # userv_list.append(user_v)
            # userid_list.append(user_id)
            # movieid_list.append(movie_id)

            # self.test_df.set_value(index=user_id, col='prediction',value=yhat)
            # if self.verbose:
            #     print(self.test_df.loc[user_id,'prediction'])

        except (KeyError, ValueError, TypeError) as err:
            print(err)
            print('Exception - Rating is dataframe mean')

    return zip(prediction_id_list, predictions)

In [15]:
somelist = [1,4,4,4,5,2,3,2,2,3,4,4,4,7]
theset = set(somelist)
theset

{1, 2, 3, 4, 5, 7}

In [17]:
l = list(theset)
l

[1, 2, 3, 4, 5, 7]

In [23]:
fn = 'hw1_datasets/dataset2/train_labels.csv'
labels2 = pd.read_csv(fn, index_col=0, header=None)
labels2.shape

(175, 1)

In [25]:
def work_chunks(df, n_chunks):
    num_rows = len(df)
    steps = math.ceil(num_rows/n_chunks) #rounding errors causes out of bounds
    return [slice(n, n+steps) for n in range(0, num_rows, steps)]  #should cover uneven steps

In [27]:
slices = work_chunks(labels2, 4)
slices

[slice(0, 44, None),
 slice(44, 88, None),
 slice(88, 132, None),
 slice(132, 176, None)]

In [33]:
l1 = labels2[slices[0]]

In [32]:
labels2.tail()

,1
0,
170,7.0
171,7.0
172,5.0
173,1.0
174,3.0


In [38]:
a = ((1,3), (3,4), (33,3),(4,5))
c =(((1,3), (3,4)), ((33,3),(4,5)))

In [37]:
b = list(zip(*a))
b

[(1, 3, 33, 4), (3, 4, 3, 5)]

In [40]:
c = list(zip(*c))
c

[((1, 3), (3, 4)), ((33, 3), (4, 5))]

In [41]:
q = [2,34,5,1,23,5]
r = [3,22,1,23,3,4]

In [43]:
t= zip(q,r)

In [44]:
list(t)

[(2, 3), (34, 22), (5, 1), (1, 23), (23, 3), (5, 4)]

In [49]:
tuple_list = [((2,3), (5,7,5), (9,2,4,2))]
pad_tuple = (0, 0, 0)
for t in tuple_list:
    t += pad_tuple             # make sure the tuple is sufficiently long
    x, y, z = t[:3]            # only extract the first three elements
    print(x,y,z)

(2, 3) (5, 7, 5) (9, 2, 4, 2)


In [51]:
list(chain.from_iterable(tuple_list))

[(2, 3), (5, 7, 5), (9, 2, 4, 2)]

In [48]:
list(zip(*tuple_list))

[(2, 5, 9), (3, 7, 2)]